In [7]:
!pip install openpyxl

  Using cached https://files.pythonhosted.org/packages/f0/da/572cbc0bc582390480bbd7c4e93d14dc46079778ed915b505dc494b37c57/jdcal-1.4.1-py2.py3-none-any.whl


In [8]:
import os
import io
import pandas as pd
from rdflib import ConjunctiveGraph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

pd.set_option('display.max_colwidth', 200)

if not os.path.exists('outputs'):
    os.makedirs('outputs')
    
g = ConjunctiveGraph()
g.parse('data.trig', format='trig')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    return pd.read_csv(io.StringIO(output.getvalue().decode('utf-8')), encoding='utf-8')
    
def write(df, fn):
    df.to_excel('outputs/{}'.format(fn), encoding='utf-8')

In [9]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_website ?ri_domain ?rep_label WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:website ?ri_website .
        ?ri envri:hasDomain ?ri_domain .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
    }
    FILTER (?date > "2019-01-01"^^xsd:date)
    # FILTER (?ri_acronym = "ICOS")
}
""")

display(q)
write(q, 'output-1.xlsx')

,date,ri_acronym,ri_website,ri_domain,rep_label
0,2019-05-03,EMSO,http://www.emso.eu,n117,EMSO data
1,2019-04-30,LW (Marine),http://www.euro-argo.eu,n6,EUROBIS
2,2019-04-30,LW (Marine),http://www.euro-argo.eu,n6,Marine Data Archive
3,2019-05-23,SDN,http://www.seadatanet.org,n151,SeaDataNet Central Data Products
4,2019-05-23,SDN,http://www.seadatanet.org,n151,SeaDataNet Common DAta Index (CDI)
5,2019-04-18,Euro-Argo,http://www.euro-argo.eu,n78,Euro-Argo Data
6,2019-04-01,ICOS,http://www.icos-cp.eu,n223,Carbon Portal


In [10]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?vocab_name ?vocab_iri WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasVocabularies [
           envri:hasName ?vocab_name ;
           envri:hasVocabularyIri ?vocab_iri 
         ]
       ]
}
""")

display(q)
write(q, 'output-2.xlsx')

,rep_label,vocab_name,vocab_iri
0,Carbon Portal,ICOS,http://meta.icos-cp.eu/ontologies/cpmeta/
1,Carbon Portal,PROV-O,http://www.w3.org/ns/prov
2,Carbon Portal,Darwin Core,http://purl.org/dc/elements/1.1/
3,EUROBIS,WoRMS,http://www.marinespecies.org/
4,EUROBIS,Marine Regions,http://www.marineregions.org/
5,EUROBIS,Marine species,http://www.marinespecies.org/
6,SeaDataNet Central Data Products,EDMO,https://edmo.seadatanet.org
7,SeaDataNet Central Data Products,NERC vocabulary service,http://vocab.nerc.ac.uk/collection/V22/current/
8,SeaDataNet Central Data Products,SeaDataNet CDI,https://cdi.seadatanet.org
9,SeaDataNet Central Data Products,ISO 19115/19139,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml


In [11]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>

SELECT ?p WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
}
""")

display(q)
write(q, 'output-3.xlsx')

,p
0,http://envri.eu/ns/hasAuthenticationMethod
1,http://envri.eu/ns/contentAccessAuthorizationRequired
2,http://envri.eu/ns/maintainsOwnUserDatabase
3,http://envri.eu/ns/usesAuthorisationTechnique


In [12]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-4.xlsx')

,l,p,o,li
0,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
1,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
2,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
3,LW (Marine) EUROBIS vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
4,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
5,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
6,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
7,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
8,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
9,LW (Marine) EUROBIS vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN


In [13]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?schema_name WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasMetadata [
           envri:hasSchema [
             envri:hasSchemaName ?schema_name
           ]
         ]
       ]
}
""")

display(q)
write(q, 'output-5.xlsx')

,rep_label,schema_name
0,Carbon Portal,http://envri.eu/ns/ISO_19115
1,Carbon Portal,http://envri.eu/ns/GeoDCAT
2,EUROBIS,http://envri.eu/ns/EML
3,EUROBIS,http://envri.eu/ns/ISO_19115
4,SeaDataNet Central Data Products,http://envri.eu/ns/ISO_19115_19139
5,SeaDataNet Central Data Products,http://envri.eu/ns/SeaDataNetCommunityProfile
6,Euro-Argo Data,http://envri.eu/ns/ArgoUserManual
7,Euro-Argo Data,http://envri.eu/ns/NetCDFCFChecker
8,SeaDataNet Common DAta Index (CDI),http://envri.eu/ns/ISO_19115_19139
9,SeaDataNet Common DAta Index (CDI),http://envri.eu/ns/SeaDataNetCommunityProfile


In [14]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I1.xlsx')

,l,p,o,li
0,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
1,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
2,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
3,LW (Marine) EUROBIS vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
4,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
5,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
6,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
7,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
8,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
9,LW (Marine) EUROBIS vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN


In [15]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1.xlsx')

,l,p,o,li
0,LW (Marine) EUROBIS data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n28,http://envri.eu/ns/RPackages
1,LW (Marine) Marine Data Archive data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/None,NaN
2,SDN SeaDataNet Common DAta Index (CDI) data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
3,EMSO EMSO data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
4,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Spark
5,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Cassandra
6,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Pangeo
7,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Elasticsearch
8,ICOS Carbon Portal data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n244,http://envri.eu/ns/JupiterNotebooks
9,SDN SeaDataNet Central Data Products data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN


In [16]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A2.xlsx')

,l,p,o,li


In [17]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_1.xlsx')

,l,p,o,li
0,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
1,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
2,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
3,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
4,ICOS Carbon Portal access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
5,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
6,EMSO EMSO data access mechanism,http://envri.eu/ns/accessWithoutCosts,true,NaN
7,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
8,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
9,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/FTP,NaN


In [18]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_2.xlsx')

,l,p,o,li
0,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/hasAuthenticationMethod,account manually,NaN
1,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/hasAuthenticationMethod,account manually,NaN
2,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/hasAuthenticationMethod,http://envri.eu/ns/VOID,NaN
3,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/hasAuthenticationMethod,Marine-ID,NaN
4,ICOS Carbon Portal access mechanism,http://envri.eu/ns/hasAuthenticationMethod,OAuth paired with eduGain,NaN
5,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/hasAuthenticationMethod,Marine-ID,NaN
6,EMSO EMSO data access mechanism,http://envri.eu/ns/hasAuthenticationMethod,http://envri.eu/ns/VOID,NaN
7,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
8,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
9,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN


In [19]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I2.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
1,EMSO EMSO data metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
2,LW (Marine) EUROBIS metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
3,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
4,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
5,Euro-Argo Euro-Argo Data metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,http://envri.eu/ns/Planned,NaN
6,ICOS Carbon Portal metadata,http://envri.eu/ns/categoriesAreDefinedInRegistries,true,NaN
7,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasVocabularyIri,http://vocab.nerc.ac.uk/collection/P01/current/PSALST01/,NaN
8,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
9,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasVocabularyIri,http://www.w3.org/ns/prov,NaN


In [20]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I3.xlsx')

,l,p,o,li


In [21]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1.xlsx')

,l,p,o,li


In [22]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_1.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/includesAccessPolicyStatements,true,NaN
1,EMSO EMSO data metadata,http://envri.eu/ns/includesAccessPolicyStatements,true,NaN
2,LW (Marine) EUROBIS metadata,http://envri.eu/ns/includesAccessPolicyStatements,partially,NaN
3,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/includesAccessPolicyStatements,partially,NaN
4,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/includesAccessPolicyStatements,true,NaN
5,Euro-Argo Euro-Argo Data metadata,http://envri.eu/ns/includesAccessPolicyStatements,true,NaN
6,ICOS Carbon Portal metadata,http://envri.eu/ns/includesAccessPolicyStatements,true,NaN
7,SDN repository,http://envri.eu/ns/hasPolicies,n180,http://envri.eu/ns/DataUsage
8,SDN repository,http://envri.eu/ns/hasPolicies,n211,http://envri.eu/ns/DataUsage
9,Euro-Argo repository,http://envri.eu/ns/hasPolicies,n107,http://envri.eu/ns/DataAccess


In [23]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_2.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN
1,EMSO EMSO data metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN
2,LW (Marine) EUROBIS metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN
3,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN
4,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN
5,ICOS Carbon Portal metadata,http://envri.eu/ns/hasMachineReadableProvenance,False,NaN


In [24]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_3.xlsx')

,l,p,o,li
0,Euro-Argo Data data management plans,http://envri.eu/ns/hasComplianceValidationService,True,NaN
1,Marine Data Archive data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
2,EUROBIS data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
3,EMSO data data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
4,SeaDataNet Central Data Products data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
5,SeaDataNet Common DAta Index (CDI) data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
6,Carbon Portal data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN


In [25]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F1.xlsx')

,l,p,o,li
0,LW (Marine) Marine Data Archive identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/LocalService,NaN
1,LW (Marine) EUROBIS identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
2,EMSO EMSO data identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/Planned,NaN
3,ICOS Carbon Portal identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
4,LW (Marine) Marine Data Archive identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
5,ICOS Carbon Portal identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/PDC,NaN
6,Euro-Argo Euro-Argo Data identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/SEANOE,NaN
7,SDN SeaDataNet Common DAta Index (CDI) identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/EUDAT,NaN
8,SDN SeaDataNet Central Data Products identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
9,LW (Marine) EUROBIS identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/LocalService,NaN


In [26]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F2.xlsx')

,l,p,o,li
0,EMSO EMSO data metadata schema,http://envri.eu/ns/includesProvenanceFields,n140,http://envri.eu/ns/TextOnly
1,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/includesProvenanceFields,http://envri.eu/ns/Planned,NaN
2,LW (Marine) EUROBIS metadata schema,http://envri.eu/ns/includesProvenanceFields,n33,http://envri.eu/ns/TextOnly
3,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/includesProvenanceFields,n209,http://envri.eu/ns/TextOnly
4,LW (Marine) Marine Data Archive metadata schema,http://envri.eu/ns/includesProvenanceFields,n63,http://envri.eu/ns/TextOnly
5,LW (Marine) Marine Data Archive metadata schema,http://envri.eu/ns/includesProvenanceFields,n65,http://envri.eu/ns/TextOnly
6,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/includesProvenanceFields,n177,http://envri.eu/ns/TextOnly
7,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/includesProvenanceFields,http://envri.eu/ns/Planned,NaN
8,ICOS Carbon Portal metadata schema,http://envri.eu/ns/includesProvenanceFields,n252,http://envri.eu/ns/TextOnly
9,Euro-Argo Euro-Argo Data metadata schema,http://envri.eu/ns/includesProvenanceFields,n104,http://envri.eu/ns/codes


In [27]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F3.xlsx')

,l,p,o,li
0,SDN repository,http://envri.eu/ns/hasMetadata,n173,NaN
1,SDN repository,http://envri.eu/ns/hasMetadata,n205,NaN
2,Euro-Argo repository,http://envri.eu/ns/hasMetadata,n99,NaN
3,EMSO repository,http://envri.eu/ns/hasMetadata,n136,NaN
4,LW (Marine) repository,http://envri.eu/ns/hasMetadata,n31,NaN
5,LW (Marine) repository,http://envri.eu/ns/hasMetadata,n61,NaN
6,ICOS repository,http://envri.eu/ns/hasMetadata,n247,NaN
7,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
8,EMSO EMSO data metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
9,LW (Marine) EUROBIS metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN


In [28]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F4 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F4.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,http://envri.eu/ns/None,NaN
1,EMSO EMSO data metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n142,http://envri.eu/ns/GeoPortal
2,EMSO EMSO data metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n142,http://envri.eu/ns/IODEOceanDataPortal
3,LW (Marine) EUROBIS metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n37,http://envri.eu/ns/GBIF
4,LW (Marine) EUROBIS metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n37,http://envri.eu/ns/EDMED
5,LW (Marine) EUROBIS metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n37,http://envri.eu/ns/DataCite
6,LW (Marine) EUROBIS metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n37,http://envri.eu/ns/GCMD
7,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,http://envri.eu/ns/Planned,NaN
8,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n179,http://envri.eu/ns/GeoPortal
9,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/supportsExternalSearchEngineTypes,n179,http://envri.eu/ns/IODEOceanDataPortal
